In [ ]:
from stc_geck.client import StcGeck
geck = StcGeck(
    ipfs_http_endpoint='http://localhost:8080',
    paths=(
        '/ipns/standard-template-construct.org/data/nexus_books/',
        '/ipns/standard-template-construct.org/data/nexus_science/',
    ),
)
# Connects to IPFS and instantiate configured indices for searching
await geck.start()

In [ ]:
# Match search in books collection
await geck.search([{
    "index_alias": "nexus_books",
    "query": {"match": {"value": "hemoglobin", "default_fields": ["title"]}},
    "collectors": [{"top_docs": {"limit": 5}}],
    "is_fieldnorms_scoring_enabled": False,
}])

In [ ]:
# Term search in science collection
await geck.search([{
    "index_alias": "nexus_science",
    "query": {"term": {"field": "doi", "value": "10.1109/healthcom54947.2022.9982758"}},
    "collectors": [{"top_docs": {"limit": 1}}],
    "is_fieldnorms_scoring_enabled": False,
}])

In [ ]:
# Complex query and count results too
SHOULD = 0
MUST = 1
MUST_NOT = 2

await geck.search([{
    "index_alias": "nexus_science",
    "query": {"boolean": {"subqueries": [
        {"occur": SHOULD, "query": {"match": {"value": "hemoglobin", "default_fields": ["title"]}}},
        {"occur": SHOULD, "query": {"match": {"value": "fetal", "default_fields": ["title"]}}},
    ]}},
    "collectors": [{"top_docs": {"limit": 5}}, {"count": {}}],
    "is_fieldnorms_scoring_enabled": False,
}])